In [1]:
import numpy as np

## Create a base case of class

- method:**__init__** to initilize all the properties, **forward** to compute the values, **backward** to propagate the error and compute the gradients

In [2]:
class Node(object):
    def __init__(self,inbound_nodes = []):
        #a list of input nodes to this node
        self.inbound_nodes = inbound_nodes
        # the output nodes of this node
        self.outbound_nodes = []
        #the eventual of this node set bu forward method
        self.value = None
        #key is input nodes,value is patial of this node with respect to input nodes
        self.gradients = {}
        #set input node's outbound node to this node
        for n in self.inbound_nodes:
            n.outbound_nodes.append(self)
            
    def forward(self):
        #every subclass need this method
        raise NotImplementedError
        
    def backward(self):
        #every subcalss need this method
        raise NotImplementedError

## Input node
- The **Input** node has no inboud nodes
- Take the gradients directly from the outbound nodes

In [3]:
class Input(Node):
    def __init__(self):
        #initialize all properties using base class
        #self.value is set during `topological_sort` later
        Node.__init__(self)
    
    def forward(self):
        pass
    
    def backward(self):
        #The Input node has no input so the the key self set to 0
        self.gradients = {self:0}
        #weight and bias may be input, get their gradients from their outbound nodes
        for n in self.outbound_nodes:
            self.gradients[self] += n.gradients[self]

## Linear node
- inbound nodes are X(training data),weights and bias
- **forward** calculate the value by numpy
- **backward** use the gradient cost from the outbound nodes to calculate the partials with respect to inbound nodes

In [4]:
class Linear(Node):
    def __init__(self,X,W,b):
        #set weights bias as inboud nodes
        Node.__init__(self,[X,W,b])
        
    def forward(self):
        X = self.inbound_nodes[0].value
        W = self.inbound_nodes[1].value
        b = self.inbound_nodes[2].value
        
        self.value = np.dot(X,W) + b
        
    def backward(self):
        self.gradients = {n:np.zeros_like(n.value) for n in self.inbound_nodes}
        
        for n in self.outbound_nodes:
            grad_cost = n.gradients[self]
            self.gradients[self.inbound_nodes[0]] += grad_cost@self.inbound_nodes[1].value.T
            self.gradients[self.inbound_nodes[1]] += self.inbound_nodes[0].value.T@grad_cost
            self.gradients[self.inbound_nodes[2]] += np.sum(grad_cost,axis=0,keepdims=False)

## Sigmoid node
- **forward** apply the sigmoid func to the input nodes
- **backward** use the gradient cost from the outbound nodes to calculate the partials with respect to inbound nodes

In [5]:
class Sigmoid(Node):
    def __init__(self,node):
        Node.__init__(self,[node])
        
    def _sigmoid(self,x):
        return 1./(1.+np.exp(-x))
    
    def forward(self):
        x = self.inbound_nodes[0].value
        self.value = self._sigmoid(x)
        
    def backward(self):
        self.gradients = {n:np.zeros_like(n.value) for n in self.inbound_nodes}
        for n in self.outbound_nodes:
            grad_cost = n.gradients[self]
            sigmoid = self.value
            self.gradients[self.inbound_nodes[0]] += grad_cost * sigmoid * (1-sigmoid)

## MSE node
- **forward** calculate the mean of squared errors
- **backward** calculate the partial with respect to the inbound nodes

In [6]:
class MSE(Node):
    def __init__(self,y,a):
        Node.__init__(self,[y,a])
        
    def forward(self):
        y = self.inbound_nodes[0].value.reshape(-1,1)
        
        a= self.inbound_nodes[1].value.reshape(-1,1)
        
        assert y.shape[0] == a.shape[0]
        
        self.m=self.inbound_nodes[0].value.shape[0]
        
        self.diff = y - a
        
        self.value = np.mean(self.diff**2)
        
    def backward(self):
        self.gradients[self.inbound_nodes[0]] = (2/self.m) * self.diff
        self.gradients[self.inbound_nodes[1]] = (-2/self.m) * self.diff
        
        

## topological sort
- create a graph according to the properties of the nodes
- sort the nodes in topological order using Kahn's Algorithm

In [7]:
def topological_sort(feed_dict):
    """
    Sort the nodes in topological order using Kahn's Algorithm.

    `feed_dict`: A dictionary where the key is a `Input` Node and the value is the respective value feed to that Node.

    Returns a list of sorted nodes.
    """

    input_nodes = [n for n in feed_dict.keys()]

    G = {}
    nodes = [n for n in input_nodes]
    while len(nodes) > 0:
        n = nodes.pop(0)
        if n not in G:
            G[n] = {'in': set(), 'out': set()}
        for m in n.outbound_nodes:
            if m not in G:
                G[m] = {'in': set(), 'out': set()}
            G[n]['out'].add(m)
            G[m]['in'].add(n)
            nodes.append(m)

    L = []
    S = set(input_nodes)
    while len(S) > 0:
        n = S.pop()

        if isinstance(n, Input):
            n.value = feed_dict[n]

        L.append(n)
        for m in n.outbound_nodes:
            G[n]['out'].remove(m)
            G[m]['in'].remove(n)
            # if no other incoming edges add to S
            if len(G[m]['in']) == 0:
                S.add(m)
    return L


## use foward and backward method according to the topological sort

In [8]:
def forward_and_backward(graph):
    # Forward pass
    for n in graph:
        n.forward()


    # Backward pass
    for n in graph[::-1]:
        n.backward()

## SGD to update weight and bias

In [9]:
def sgd_update(trainables, learning_rate=1e-2):
    """
    Updates the value of each trainable with SGD.

    Arguments:

        `trainables`: A list of `Input` Nodes representing weights/biases.
        `learning_rate`: The learning rate.
    """
    # Performs SGD
    #
    # Loop over the trainables
    for t in trainables:
        # Change the trainable's value by subtracting the learning rate
        # multiplied by the partial of the cost with respect to this
        # trainable.
        partial = t.gradients[t]
        t.value -= learning_rate * partial

## Predict the Boston house price using miniflow

In [10]:
"""
Have fun with the number of epochs!

Be warned that if you increase them too much,
the VM will time out :)
"""

import numpy as np
from sklearn.datasets import load_boston
from sklearn.utils import shuffle, resample
#from miniflow import *

# Load data
data = load_boston()
X_ = data['data']
y_ = data['target']

# Normalize data
X_ = (X_ - np.mean(X_, axis=0)) / np.std(X_, axis=0)

n_features = X_.shape[1]
n_hidden = 10
W1_ = np.random.randn(n_features, n_hidden)
b1_ = np.zeros(n_hidden)
W2_ = np.random.randn(n_hidden, 1)
b2_ = np.zeros(1)

# Neural network
X, y = Input(), Input()
W1, b1 = Input(), Input()
W2, b2 = Input(), Input()

l1 = Linear(X, W1, b1)
s1 = Sigmoid(l1)
l2 = Linear(s1, W2, b2)
cost = MSE(y, l2)

feed_dict = {
    X: X_,
    y: y_,
    W1: W1_,
    b1: b1_,
    W2: W2_,
    b2: b2_
}

epochs = 1000
# Total number of examples
m = X_.shape[0]
batch_size = 11
steps_per_epoch = m // batch_size

graph = topological_sort(feed_dict)
trainables = [W1, b1, W2, b2]

print("Total number of examples = {}".format(m))

# Step 4
for i in range(epochs):
    loss = 0
    for j in range(steps_per_epoch):
        # Step 1
        # Randomly sample a batch of examples
        X_batch, y_batch = resample(X_, y_, n_samples=batch_size)

        # Reset value of X and y Inputs
        X.value = X_batch
        y.value = y_batch

        # Step 2
        forward_and_backward(graph)

        # Step 3
        sgd_update(trainables)

        loss += graph[-1].value
        

    print("Epoch: {}, Loss: {:.3f}".format(i+1, loss/steps_per_epoch))


Total number of examples = 506
Epoch: 1, Loss: 133.238
Epoch: 2, Loss: 31.995
Epoch: 3, Loss: 23.921
Epoch: 4, Loss: 27.718
Epoch: 5, Loss: 18.915
Epoch: 6, Loss: 21.028
Epoch: 7, Loss: 19.623
Epoch: 8, Loss: 18.517
Epoch: 9, Loss: 14.931
Epoch: 10, Loss: 13.225
Epoch: 11, Loss: 15.511
Epoch: 12, Loss: 10.678
Epoch: 13, Loss: 9.544
Epoch: 14, Loss: 13.647
Epoch: 15, Loss: 12.702
Epoch: 16, Loss: 10.774
Epoch: 17, Loss: 13.872
Epoch: 18, Loss: 12.359
Epoch: 19, Loss: 9.636
Epoch: 20, Loss: 12.539
Epoch: 21, Loss: 10.275
Epoch: 22, Loss: 11.044
Epoch: 23, Loss: 11.919
Epoch: 24, Loss: 10.690
Epoch: 25, Loss: 10.784
Epoch: 26, Loss: 10.346
Epoch: 27, Loss: 8.929
Epoch: 28, Loss: 9.555
Epoch: 29, Loss: 10.761
Epoch: 30, Loss: 8.754
Epoch: 31, Loss: 9.735
Epoch: 32, Loss: 9.734
Epoch: 33, Loss: 10.091
Epoch: 34, Loss: 9.159
Epoch: 35, Loss: 12.140
Epoch: 36, Loss: 9.680
Epoch: 37, Loss: 8.967
Epoch: 38, Loss: 7.741
Epoch: 39, Loss: 9.547
Epoch: 40, Loss: 8.276
Epoch: 41, Loss: 9.580
Epoch: 

Epoch: 349, Loss: 3.981
Epoch: 350, Loss: 3.904
Epoch: 351, Loss: 4.086
Epoch: 352, Loss: 3.795
Epoch: 353, Loss: 3.882
Epoch: 354, Loss: 3.135
Epoch: 355, Loss: 4.158
Epoch: 356, Loss: 4.051
Epoch: 357, Loss: 3.544
Epoch: 358, Loss: 3.701
Epoch: 359, Loss: 3.798
Epoch: 360, Loss: 3.732
Epoch: 361, Loss: 3.825
Epoch: 362, Loss: 3.528
Epoch: 363, Loss: 3.670
Epoch: 364, Loss: 3.649
Epoch: 365, Loss: 3.244
Epoch: 366, Loss: 3.669
Epoch: 367, Loss: 3.908
Epoch: 368, Loss: 3.559
Epoch: 369, Loss: 3.521
Epoch: 370, Loss: 4.102
Epoch: 371, Loss: 3.421
Epoch: 372, Loss: 3.578
Epoch: 373, Loss: 3.147
Epoch: 374, Loss: 4.013
Epoch: 375, Loss: 3.735
Epoch: 376, Loss: 4.038
Epoch: 377, Loss: 3.611
Epoch: 378, Loss: 3.615
Epoch: 379, Loss: 3.519
Epoch: 380, Loss: 3.740
Epoch: 381, Loss: 3.222
Epoch: 382, Loss: 3.929
Epoch: 383, Loss: 4.039
Epoch: 384, Loss: 4.075
Epoch: 385, Loss: 3.804
Epoch: 386, Loss: 4.082
Epoch: 387, Loss: 3.444
Epoch: 388, Loss: 3.807
Epoch: 389, Loss: 3.410
Epoch: 390, Loss

Epoch: 697, Loss: 3.493
Epoch: 698, Loss: 3.581
Epoch: 699, Loss: 3.015
Epoch: 700, Loss: 3.526
Epoch: 701, Loss: 3.579
Epoch: 702, Loss: 3.536
Epoch: 703, Loss: 3.072
Epoch: 704, Loss: 3.687
Epoch: 705, Loss: 3.642
Epoch: 706, Loss: 3.239
Epoch: 707, Loss: 3.083
Epoch: 708, Loss: 3.401
Epoch: 709, Loss: 3.007
Epoch: 710, Loss: 3.561
Epoch: 711, Loss: 3.657
Epoch: 712, Loss: 3.479
Epoch: 713, Loss: 3.421
Epoch: 714, Loss: 3.480
Epoch: 715, Loss: 3.755
Epoch: 716, Loss: 3.015
Epoch: 717, Loss: 3.314
Epoch: 718, Loss: 3.514
Epoch: 719, Loss: 3.273
Epoch: 720, Loss: 3.284
Epoch: 721, Loss: 3.474
Epoch: 722, Loss: 3.127
Epoch: 723, Loss: 3.074
Epoch: 724, Loss: 3.386
Epoch: 725, Loss: 3.390
Epoch: 726, Loss: 3.591
Epoch: 727, Loss: 3.310
Epoch: 728, Loss: 3.281
Epoch: 729, Loss: 3.778
Epoch: 730, Loss: 3.160
Epoch: 731, Loss: 3.459
Epoch: 732, Loss: 3.563
Epoch: 733, Loss: 3.481
Epoch: 734, Loss: 3.370
Epoch: 735, Loss: 3.606
Epoch: 736, Loss: 3.286
Epoch: 737, Loss: 3.477
Epoch: 738, Loss